### 301 형식, 구어체로만 구성된 폴더 처리

In [1]:
import sys

import docx
import os
from datetime import datetime

import re
import win32com.client
import pandas as pd
import pymysql
from pymysql import MySQLError

def get_file_info(file_path):
    file_size = os.path.getsize(file_path)                  # 파일 크기를 바이트 단위로 가져오기 
    file_creation_time = os.path.getctime(file_path)        # 파일 생성일자를 가져오기
    file_modification_time = os.path.getmtime(file_path)    # 파일 최종 수정일자를 가져오기
    # file_name = os.path.basename(file_path)                 # 파일명 추출
    file_name, file_extension = os.path.splitext(os.path.basename(file_path))                 # 확장명 뺀 파일명 추출
    
    return file_name, file_size, file_creation_time, file_modification_time

def format_time(timestamp):
    dt_object = datetime.fromtimestamp(timestamp)               # 에포크 시간을 datetime 객체로 변환
    formatted_time = dt_object.strftime('%Y-%m-%d %H:%M:%S')    # 원하는 형식으로 시간을 포맷팅
    
    return formatted_time

def count_words_docx(file_path):
    total_words = 0
    doc = docx.Document(file_path)

    for paragraph in doc.paragraphs:
        total_words += len(paragraph.text.split())   # 문서 내 단어 수 세기 

    return total_words

In [3]:
# 파일 복제
import shutil

input_file = '2023-12-06 구어체'
input_root = os.path.join('Z:/', input_file)

# shutil.copytree('복사 대상 위치', '붙여넣기할 대상 위치')
# 기본적으로 2번째 인자 폴더를 makedir하기 때문에 마지막 하위 폴더명이 존재하면 안됨.
if __name__ == '__main__':
    shutil.copytree(
        # 'Z:/2023-11-15 구어체/',
        input_root,
        # 'Y:/구어체_모음' -- 이 폴더가 이미 존재하기 때문에 처리가 안됨.
        os.path.join('Y:/구어체_모음', input_file)
    )

In [3]:
# MySQL 연결 정보
mysql_host = '172.30.1.36'
mysql_port = 13333
mysql_user = 'vnc'
mysql_password = 'vnc'
mysql_database = 'vnc'


excel_tb = 'load_vnc_org_lst_sim_20231207_05'
file_tb = 'load_vnc_file_list_sim_20231207_05'
docx_tb = 'load_vnc_file_docx_sim_20231207_05'

In [4]:
# 테이블 생성 쿼리

# 테이블 생성하는 쿼리문 결과
CREATE_TABLE = False  # 기본적으로 fALSE로 초기화

try:    
    db = pymysql.connect(host=mysql_host,port=mysql_port,user=mysql_user,passwd=mysql_password,db=mysql_database)
    cursor = db.cursor(pymysql.cursors.DictCursor)
    print("Connected to MySQL database for create table.")
    
    create_query = f'CREATE TABLE {excel_tb} LIKE vnc.load_vnc_org_lst_sim;'
    cursor.execute(create_query)    # 쿼리 실행

    create_query = f'CREATE TABLE {file_tb} LIKE vnc.load_vnc_file_list_sim;'
    cursor.execute(create_query)

    create_query = f'CREATE TABLE {docx_tb} LIKE vnc.load_vnc_file_docx_sim;'
    cursor.execute(create_query)
   
    db.commit() # 변경사항 DB에 반영
    
    create_idx_q = f'CREATE INDEX {docx_tb}_FILE_TXT_IDX USING BTREE ON {docx_tb} (FILE_TXT);'
    cursor.execute(create_idx_q)    # 쿼리 실행
    db.commit() # 변경사항 DB에 반영
    
    print("Create succeeded.")
    CREATE_TABLE = True
    
except Exception as e:
    print(f"에러 발생: {e}")
    sql = ''
    sql += f'DROP TABLE IF EXISTS {excel_tb};'
    cursor.execute(sql)
    
    sql = 'DROP TABLE IF EXISTS {file_tb};'
    cursor.execute(sql)
    
    sql = 'DROP TABLE IF EXISTS {docx_tb};'
    cursor.execute(sql)

finally:
    db.close()
    cursor.close()
    print(f"DB is disconnected.")

Connected to MySQL database for create table.
Create succeeded.
DB is disconnected.


In [29]:
root_path = 'Y:/구어체_모음/2023-11-27 구어체/'

JOB_YMD = ''
WORKER_ID = ''

worker_folders = os.listdir(root_path)
for worker in worker_folders:
    WORKER_ID = worker
    
    if len(WORKER_ID) > 5 :
        # 작업자 폴더 길이가 5를 초과하면 해당 파일 처리를 건너뜀.(자동화로 처리된 파일)
        continue
    
    worker_path = os.path.join(root_path, worker)
    
    job_folder = os.listdir(worker_path)[0]  # 가장 첫번째 폴더 가져옴
    job_folder_path = os.path.join(worker_path, job_folder)
    
    JOB_YMD = os.path.basename(job_folder_path) # 가장 마지막 경로명 추출
    # WORKER_ID = os.path.dirname(job_folder_path)    # 가장 마지막 경로명의 상위 경로 추출 (Y:/구어체_모음/2023-11-15 구어체/CW040)
    print(WORKER_ID)
    
    
    

CW040
CW044
CW045
CW046
CW048
CW049
CW050
CW051


In [21]:
# 작업자 작업경로
# root_path = 'Y:/구어체_모음/2023-11-10 구어체/CW040/20231110/'
# root_path = 'Y:/구어체_모음/2023-11-10 구어체/CW044/20231110-1/'
# root_path = 'Y:/구어체_모음/2023-11-10 구어체/CW046/20231110-1/'
# root_path = 'Y:/구어체_모음/2023-11-10 구어체/CW047/20231110-1/'
# root_path = 'Y:/구어체_모음/2023-11-10 구어체/CW048/20231110/'
# root_path = 'Y:/구어체_모음/2023-11-10 구어체/CW049/20231110/'
# root_path = 'Y:/구어체_모음/2023-11-10 구어체/CW050/20231110-1/'
# root_path = 'Y:/구어체_모음/2023-11-10 구어체/CW051/20231110-1/'
# root_path = 'Y:/구어체_모음/2023-11-10 구어체/CW044/20231110-2/'
# root_path = 'Y:/구어체_모음/2023-11-10 구어체/CW046/20231110-2/'
# root_path = 'Y:\\구어체_모음\\2023-11-10 구어체\\CW047\\20231110-2/'
# root_path = 'Y:\\구어체_모음\\2023-11-10 구어체\\CW050\\20231110-2/'
# root_path = 'Y:\\구어체_모음\\2023-11-10 구어체\\CW051\\20231110-2/'
# root_path = 'Y:\구어체_모음\\2023-11-13 구어체\\CW040\\20231113/'
# root_path = f'Y:\구어체_모음\\2023-11-13 구어체\CW044\\20231113/'
# root_path = f'Y:\구어체_모음\\2023-11-13 구어체\CW045\\20231113/'
# root_path = f'Y:\구어체_모음\\2023-11-13 구어체\CW046\\20231113-1/'
# root_path = f'Y:\구어체_모음\\2023-11-13 구어체\CW047\\20231113/'
# root_path = f'Y:\구어체_모음\\2023-11-13 구어체\CW048\\20231113/'
# root_path = f'Y:\구어체_모음\\2023-11-13 구어체\CW049\\20231113/'
# root_path = f'Y:\구어체_모음\\2023-11-13 구어체\\CW050\\20231113/'
# root_path = f'Y:\구어체_모음\\2023-11-13 구어체\CW051\\20231113/'
# root_path = f'Y:\\구어체_모음\\2023-11-14 구어체\\CW040\\20231114/'
# root_path = f'Y:\\구어체_모음\\2023-11-14 구어체\\CW044\\20231114/'
# root_path = f'Y:\\구어체_모음\\2023-11-14 구어체\\CW045\\20231114/'
# root_path = f'Y:\\구어체_모음\\2023-11-14 구어체\\CW046\\20231114/'
# root_path = f'Y:\\구어체_모음\\2023-11-14 구어체\\CW047\\20231114/'   # CW047	20231114 SEQ 1033 ~ 1080
# root_path = f'Y:\\구어체_모음\\2023-11-14 구어체\\CW048\\20231114/'   
# root_path = f'Y:\\구어체_모음\\2023-11-14 구어체\\CW049\\20231114/'   
# root_path = f'Y:\\구어체_모음\\2023-11-14 구어체\\CW050\\20231114/'   
# root_path = f'Y:\\구어체_모음\\2023-11-14 구어체\\CW051\\20231114/'   
# root_path = f'Y:\\구어체_모음\\2023-11-15 구어체\\CW051\\20231115/'   
# root_path = f'Y:\\구어체_모음\\2023-11-20 구어체\\CW040\\20231120/'   
# root_path = f'Y:\\구어체_모음\\2023-11-21 구어체\\CW051\\20231121/'  

In [5]:
# 작업자 작업경로(위가 너무 길어 분할)
# root_path = r'Y:\구어체_모음\2023-11-22 구어체\CW051\20231122/'       # r : raw. 문자열 앞에 'r'을 붙이면 해당 문자열이 'raw string'으로 처리되어 이스케이프 문자를 해석하지 않고 그대로 문자열로 인식함!! -> 파일 경로 문자열에서 백슬래쉬를 이스케이프 문자로 처리하지 않고 그대로 사용할 수 있음.  
# root_path = f'Y:\\구어체_모음\\2023-11-22 구어체\\CW051\\20231122/'   # f : f-string. Python 3.6 이상에서 도입된 문자열 포맷팅 방법. 문자열 안에서 변수나 표현식을 {}로 감싸 사용햘 수 있음.
# root_path = f'Y:\\구어체_모음\\2023-11-23 구어체\\CW049\\20231123/'  
# root_path = f'Y:\\구어체_모음\\2023-11-24 구어체\\CW051\\20231124/'  
# root_path = f'Y:\\구어체_모음\\2023-11-28 구어체\\CW049/'  
# root_path = f'Y:\\구어체_모음\\2023-11-28 구어체\\CW051\\20231128/'  
# root_path = f'Y:\\구어체_모음\\2023-11-30 구어체\\CW045/'  
# root_path = f'Y:\\구어체_모음\\2023-11-30 구어체\\CW047\\20231130/'  # 46, 47, 50, 51
# root_path = f'Y:\\구어체_모음\\2023-12-01 구어체\\20231201/'  
# root_path = f'Y:\\구어체_모음\\2023-12-04 구어체\\20231204/'  
# root_path = f'Y:\\구어체_모음\\2023-12-05 구어체\\20231205/'  
# root_path = f'Y:\\구어체_모음\\2023-12-06 구어체/'  
root_path = f'Y:\\구어체_모음\\2023-12-07 구어체/'  

In [7]:
# 작업자 폴더 하나씩 잡아서 처리

# 처리
# JOB_YMD = '20231130'
# WORKER_ID = os.path.basename(root_path[:-1])

JOB_YMD = os.path.basename(root_path[:-1])  # 마지막 '/'을 뺀 디렉토리 경로를 넣어줘야 함.  JOB_YMD : 2023-12-06 구어체
# JOB_YMD = JOB_YMD[:4] + JOB_YMD[5:7] + JOB_YMD[8:10]

# WORKER_ID = os.path.basename(root_path[:-10])   # root_path가 문자열형이므로 \\ -> /로 처리되므로 1개로 생각! 
WORKER_ID = 'CW047'
print(root_path, JOB_YMD, WORKER_ID)

files = [file for file in os.listdir(root_path) if not os.path.basename(os.path.join(root_path, file)).startswith('~$') 
         and (os.path.basename(os.path.join(root_path, file)).endswith('.docx') or os.path.basename(os.path.join(root_path, file)).endswith('.xlsx'))]

WORK_CNT = len(files) - 1    # docx 파일 개수
        
# 테이블이 정상적으로 CREATE 된 경우
# if CREATE_TABLE:
if True:
    
    try:   
        db = pymysql.connect(host=mysql_host,port=mysql_port,user=mysql_user,passwd=mysql_password,db=mysql_database)
        cursor = db.cursor(pymysql.cursors.DictCursor)
        print("Connected to MySQL database")
        
        excel_bulk_insert_datas = []
        file_bulk_insert_datas = []
        # docx_bulk_insert_datas = []
            
        for file in files:

            # print(f'{file} 처리 시작')
            # print(f'\tJOB_YMD : {JOB_YMD}')
            # print(f'\tWORKER_ID : {WORKER_ID}')
            # print(f'\tWORK_CNT : {WORK_CNT}')
            
            file_path = root_path+file
            
            # 파일 정보 가져오기
            FILE_NAME, FILE_SIZE, file_creation_time, file_modification_time = get_file_info(file_path)

            # 시간을 원하는 형식으로 변환
            FILE_CREATETIME = format_time(file_creation_time)
            FILE_CHANG_TIME = format_time(file_modification_time)

            # 단어수 업데이트 할 때
            if file_path.lower().endswith(".docx"):
                doc = docx.Document(file_path)
                SEQ = 1
                WORD_COUNT = 0
                docx_bulk_insert_datas = []
                
                for paragraph in doc.paragraphs:
                    WORD_COUNT += len(paragraph.text.split())   # 문서 내 단어 수 세기 

                    texts = paragraph.text.split('\n')
                    texts = [txts for txts in texts if not txts == '' ]           # 빈공백으로만 있는 텍스트 제거
                    texts = [txts for txts in texts if not txts.strip() == '' ]   # 앞뒤 공백 자른 txts가 ''인 것 제거
                    
                    # '\n' 별로 저장할 때           
                    if texts:
                        for text in texts:
                            text = text.replace("'", "''")
                            text = text.replace("\u00A0", " ")    # strip()이 걸러내지 못하는 &nbsp 제거
                            text = text.replace("\uFEFF", " ")    # HTML 코드(&#65279;) 제거
                            text = text.replace('\t', ' ')
                            text = text.replace('\xa0', ' ')
                            text = text.strip()     # 앞 뒤 공백 제거
                            # print(f'{SEQ} : {text} \n')
                            
                            # 벌크에 넣기
                            docx_bulk_insert_datas.append((FILE_NAME, SEQ, text))

                            # text line_seq 증가
                            SEQ += 1
                    
                sql = ''
                sql = 'SAVEPOINT a'
                cursor.execute(sql)
                
                # 한 파일당 인서트             
                # .docx 파일 정보 인서트
                sql = ""
                sql =  sql + f" INSERT INTO {file_tb} "
                sql = sql + " (workymd, worker_id, workcnt, file_name, file_size, file_createtime, file_chang_time, word_count) "
                sql = sql + f" VALUES ('{JOB_YMD}', '{WORKER_ID}', '{WORK_CNT}', '{FILE_NAME}', '{FILE_SIZE}', '{FILE_CREATETIME}', '{FILE_CHANG_TIME}', '{WORD_COUNT}') "
                cursor.execute(sql)
                # db.commit()
                
                # .docx 문서 문단 인서트
                sql = ""
                sql = sql + f" INSERT INTO {docx_tb} (FILE_NAME, SEQ, FILE_TXT) VALUES (%s, %s, %s)"
                cursor.executemany(sql, docx_bulk_insert_datas)
                # db.commit()
                
                print("file_name : ", FILE_NAME)
                # print(f"{file} 인서트 완료")


            if file_path.endswith(".xlsx"):
                
                # 엑셀에서 데이터 GET
                # bulk_insert_datas = []
                datas = []
                read_flag = False

                data_frame = pd.read_excel(file_path, engine='openpyxl')
                datas.append(data_frame)

                # chunk_size = 300  # 원하는 청크 크기
                
                for df in datas:
                    for index, row in df.iterrows():
                        # print('row',row)
                        # print('row[0]',row[0])
                    
                        if(str(row.iloc[0]) == '1'):
                            read_flag = True
                        
                        if(str(row.iloc[0]) == ''):
                            read_flag = False
                        
                        if(read_flag):
                            SEQ = int(row.iloc[0])
                            PUB_YMD = str(row.iloc[3]).strip()
                            WTR_KR = str(row.iloc[4]).replace("'", "''").strip()
                            WTR_VN = str(row.iloc[5]).replace("'", "''").strip()
                            CTR_KR = str(row.iloc[6]).replace("'", "''").strip()
                            CTR_VN = str(row.iloc[7]).replace("'", "''").strip()
                            ORG_TYP = str(row.iloc[8]).strip()
                            DAT_SRC = str(row.iloc[9]).replace("'", "''").strip()   # 작은따옴표를 두 번 사용하여 이스케이프
                            DAT_TYP = str(row.iloc[10]).strip()
                            TOPIC_KR = str(row.iloc[11]).strip()
                            TOPIC_VN = str(row.iloc[12]).strip()
                            TITLE = str(row.iloc[13]).replace("'", "''").strip()  # 작은따옴표를 두 번 사용하여 이스케이프
                            COL_PK = str(row.iloc[14]).split('.')[0]
                            # DOC_STYLE = '구어체' if '구어체 여부' in str(row.iloc[15]).strip() else '문어체'
                            DOC_STYLE = '구어체'
                            # print(SEQ, COL_PK, PUB_YMD, TITLE)
                                                        
                            # {excel_tb}에 인서트
                            sql = ""
                            sql = sql + f" INSERT INTO {excel_tb} "
                            sql = sql + " (COL_PK, SEQ, WORKER_ID, JOB_YMD, PUB_YMD, WTR_KR, WTR_VN, CTR_KR, CTR_VN, ORG_TYP, DAT_SRC, DAT_TYP, TOPIC_KR, TOPIC_VN, TITLE, FILE_NAME, DOC_STYLE) "
                            sql = sql + f" VALUES ('{COL_PK}', '{SEQ}', '{WORKER_ID}', '{JOB_YMD}', '{PUB_YMD}', '{WTR_KR}', '{WTR_VN}', '{CTR_KR}', '{CTR_VN}', '{ORG_TYP}', '{DAT_SRC}', '{DAT_TYP}', '{TOPIC_KR}', '{TOPIC_VN}', '{TITLE}', '{COL_PK}', '{DOC_STYLE}') "
                            cursor.execute(sql)
                            # db.commit()
                print(f'{file} 완료! ')
        
        # for문 다 돈 후 db 영구반영         
        db.commit() 

    except Exception as e:
        print(f"Error at preceeding {file}")
        print(f"Error: {e}")
        
        # sql = ''
        # sql = 'rollback to a'
        # cursor.execute(sql)
        
        sys.exit()

    finally:
        if cursor:
            cursor.close()
        if db:
            db.close()
            print("Disconnected from MySQL database \n")

Y:\구어체_모음\2023-12-05 구어체\20231205/ 20231205 CW047
Connected to MySQL database
file_name :  05_CW047_20231205_0001
file_name :  05_CW047_20231205_0002
file_name :  05_CW047_20231205_0003
file_name :  05_CW047_20231205_0004
file_name :  05_CW047_20231205_0005
file_name :  05_CW047_20231205_0006
file_name :  05_CW047_20231205_0007
file_name :  05_CW047_20231205_0008
file_name :  05_CW047_20231205_0009
file_name :  05_CW047_20231205_0010
file_name :  05_CW047_20231205_0011
file_name :  05_CW047_20231205_0012
file_name :  05_CW047_20231205_0013
file_name :  05_CW047_20231205_0014
file_name :  05_CW047_20231205_0015
file_name :  05_CW047_20231205_0016
file_name :  05_CW047_20231205_0017
file_name :  05_CW047_20231205_0018
file_name :  05_CW047_20231205_0019
file_name :  05_CW047_20231205_0020
file_name :  05_CW047_20231205_0021
file_name :  05_CW047_20231205_0022
file_name :  05_CW047_20231205_0023
file_name :  05_CW047_20231205_0024
file_name :  05_CW047_20231205_0025
file_name :  05_CW047_

In [7]:
# 작업자 폴더 자동으로 잡아서 처리

# 한 작업자가 작업한 양을 저장할 배열 초기화
excel_bulk_insert_datas = []
file_bulk_insert_datas = []
docx_bulk_insert_datas = []

JOB_YMD = os.path.basename(root_path[:-1])  # 마지막 '/'을 뺀 디렉토리 경로를 넣어줘야 함.  JOB_YMD : 2023-12-06 구어체
JOB_YMD = JOB_YMD[:4] + JOB_YMD[5:7] + JOB_YMD[8:10]

# 기록
now = datetime.today().strftime('%Y%m%d%H%M%S')
sys.stdout = open(f'C:\\YJ\\Python_print/{now}.txt','w', encoding="utf-8")  # 현재 위치에 파일 생성, 기록

try:
    #  MySQL 데이터베이스에 연결
    db = pymysql.connect(host=mysql_host,port=mysql_port,user=mysql_user,passwd=mysql_password,db=mysql_database)   
    cursor = db.cursor(pymysql.cursors.DictCursor)  # 데이터베이스와 상호작용하기 위한 커서(cursor)를 생성
    print("Connected to MySQL database")
    
    worker_folders = os.listdir(root_path)
    for worker in worker_folders:
        WORKER_ID = worker
        
        if len(WORKER_ID) > 5 :
            # 작업자 폴더 길이가 5를 초과하면 해당 파일 처리를 건너뜀.(자동화로 처리된 파일)
            continue
        
        worker_path = os.path.join(root_path, worker)   # Y:\구어체_모음\2023-12-06 구어체\CW040
        
        # 작업자 폴더 하위에 JOB_YMD 폴더가 따로 존재하는 경우(ex. Y:\구어체_모음\2023-11-22 구어체\CW040\20231122\.docx)
        # job_folder = os.listdir(worker_path)[0]  # 가장 첫번째 폴더 가져옴
        # job_folder_path = os.path.join(worker_path, job_folder)
        
        # JOB_YMD = os.path.basename(job_folder_path) # 가장 마지막 경로명 추출
        # WORKER_ID = os.path.dirname(job_folder_path)    # 가장 마지막 경로명의 상위 경로 추출 (Y:/구어체_모음/2023-11-15 구어체/CW040)
        
        print(JOB_YMD, WORKER_ID + ' 처리 시작')
        
        # 작업자의 .docx, .xlsx에 접근
        files = [file for file in os.listdir(worker_path) if not os.path.basename(os.path.join(worker_path, file)).startswith('~$') 
            and (os.path.basename(os.path.join(worker_path, file)).endswith('.docx') or os.path.basename(os.path.join(worker_path, file)).endswith('.xlsx'))]

        WORK_CNT = len(files) - 1    # docx 파일 개수
                
        # 테이블이 정상적으로 CREATE 된 경우
        # if CREATE_TABLE:
        if True:
            
            # 초기화
            excel_bulk_insert_datas = []
            file_bulk_insert_datas = []
            docx_bulk_insert_datas = []
                
            for file in files:
                
                file_path = os.path.join(worker_path, file)
            
                # .docx 처리
                if file_path.lower().endswith(".docx"):
                    
                    # {file_db} 
                    # 파일 정보 가져오기
                    FILE_NAME, FILE_SIZE, file_creation_time, file_modification_time = get_file_info(file_path)

                    # 시간을 원하는 형식으로 변환
                    FILE_CREATETIME = format_time(file_creation_time)
                    FILE_CHANG_TIME = format_time(file_modification_time)
                    
                    # {docx_tb} 처리
                    doc = docx.Document(file_path)
                    SEQ = 1
                    WORD_COUNT = 0
                    # docx_bulk_insert_datas = []
                    
                    for paragraph in doc.paragraphs:
                        WORD_COUNT += len(paragraph.text.split())   # 문서 내 단어 수 세기 

                        texts = paragraph.text.split('\n')
                        texts = [txts for txts in texts if not txts == '' ]           # 빈공백으로만 있는 텍스트 제거
                        texts = [txts for txts in texts if not txts.strip() == '' ]   # 앞뒤 공백 자른 txts가 ''인 것 제거
                        
                        # '\n' 별로 저장할 때           
                        if texts:
                            for text in texts:
                                text = text.replace("'", "''")         # 디비 인서트를 위한 처리
                                text = text.replace("\u00A0", " ")    # strip()이 걸러내지 못하는 &nbsp 제거
                                text = text.replace("\uFEFF", " ")    # HTML 코드(&#65279;) 제거
                                text = text.replace('\t', ' ')
                                text = text.replace('\xa0', ' ')
                                text = text.strip()     # 앞 뒤 공백 제거
                                # print(f'{SEQ} : {text} \n')
                                
                                # 벌크에 넣기
                                docx_bulk_insert_datas.append((FILE_NAME, SEQ, text))

                                # text line_seq 증가
                                SEQ += 1
                    
                    file_bulk_insert_datas.append((JOB_YMD, WORKER_ID, WORK_CNT, FILE_NAME, FILE_SIZE, FILE_CREATETIME, FILE_CHANG_TIME, WORD_COUNT))
                    print(f'{FILE_NAME} fin')

                elif file_path.endswith(".xlsx"):
                    
                    # 엑셀에서 데이터 GET
                    # bulk_insert_datas = []
                    datas = []
                    read_flag = False

                    data_frame = pd.read_excel(file_path, engine='openpyxl')
                    datas.append(data_frame)

                    # chunk_size = 300  # 원하는 청크 크기
                    
                    for df in datas:
                        for index, row in df.iterrows():
                            # print('row',row)
                            # print('row[0]',row[0])
                        
                            if(str(row.iloc[0]) == '1'):
                                read_flag = True
                            
                            if(str(row.iloc[0]) == ''):
                                read_flag = False
                            
                            if(read_flag):
                                SEQ = int(row.iloc[0])
                                PUB_YMD = str(row.iloc[3]).strip()
                                WTR_KR = str(row.iloc[4]).replace("'", "''").strip()
                                WTR_VN = str(row.iloc[5]).replace("'", "''").strip()
                                CTR_KR = str(row.iloc[6]).replace("'", "''").strip()
                                CTR_VN = str(row.iloc[7]).replace("'", "''").strip()
                                ORG_TYP = str(row.iloc[8]).strip()
                                DAT_SRC = str(row.iloc[9]).replace("'", "''").strip()   # 작은따옴표를 두 번 사용하여 이스케이프
                                DAT_TYP = str(row.iloc[10]).strip()
                                TOPIC_KR = str(row.iloc[11]).strip()
                                TOPIC_VN = str(row.iloc[12]).strip()
                                TITLE = str(row.iloc[13]).replace("'", "''").strip()  # 작은따옴표를 두 번 사용하여 이스케이프
                                COL_PK = str(row.iloc[14]).split('.')[0]
                                # DOC_STYLE = '구어체' if '구어체 여부' in str(row.iloc[15]).strip() else '문어체'
                                DOC_STYLE = '구어체'
                                # print(SEQ, COL_PK, PUB_YMD, TITLE)
                                                            
                                # {excel_tb}에 인서트
                                excel_bulk_insert_datas.append((COL_PK, SEQ, WORKER_ID, JOB_YMD, PUB_YMD, WTR_KR, 
                                                                WTR_VN, CTR_KR, CTR_VN, ORG_TYP, DAT_SRC, DAT_TYP, 
                                                                TOPIC_KR, TOPIC_VN, TITLE, COL_PK, DOC_STYLE))
                                
                    print(f'{file} fin')
                
            # 작업자별로 인서트    
            sql = ''
            sql = f'SAVEPOINT {WORKER_ID}'
            cursor.execute(sql)
            
            # {file_tb} 인서트
            sql = ""
            sql = sql + f" INSERT INTO {file_tb} (workymd, worker_id, workcnt, file_name, file_size, file_createtime, file_chang_time, word_count)"
            sql = sql + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.executemany(sql, file_bulk_insert_datas)
            # db.commit()
            
            # {docx_tb} 300개씩 인서트
            for i in range(0, len(docx_bulk_insert_datas), 300):
                batch = docx_bulk_insert_datas[i:i+300]
                
                sql = ""
                sql = sql + f" INSERT INTO {docx_tb} (FILE_NAME, SEQ, FILE_TXT)"
                sql = sql + " VALUES (%s, %s, %s)"
                cursor.executemany(sql, batch)
                # db.commit()   
                # print(f'{len(batch)}')
            
            # {excel_tb} 인서트
            sql = ""
            sql = sql + f" INSERT INTO {excel_tb} "
            sql = sql + " (COL_PK, SEQ, WORKER_ID, JOB_YMD, PUB_YMD, WTR_KR, WTR_VN, CTR_KR, CTR_VN, ORG_TYP, DAT_SRC, DAT_TYP, TOPIC_KR, TOPIC_VN, TITLE, FILE_NAME, DOC_STYLE) "
            sql = sql + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) "
            cursor.executemany(sql, excel_bulk_insert_datas)
            # db.commit()
            
            # 3개의 테이블 모두 execute 후 db 영구반영         
            db.commit()    
            
            print(f"{WORKER_ID} FIN")

except Exception as e:
    print(f"Error at preceeding {WORKER_ID} files")
    print(f"Error: {e}")
    
    sql = ''
    sql = f'rollback to {WORKER_ID}'
    cursor.execute(sql)
    
    sys.exit()

finally:
    if cursor:
        cursor.close()
    if db:
        db.close()
        print("Disconnected from MySQL database \n")

if sys:
    sys.stdout.close()
    